# 🎯 IEEE-CIS Fraud Detection: Baseline модели

## Цель
Построить **baseline модели** с дефолтными параметрами для оценки качества детекции мошеннических транзакций. Это отправная точка перед оптимизацией.

## Датасет
- **Источник**: [IEEE-CIS Fraud Detection (Kaggle)](https://www.kaggle.com/c/ieee-fraud-detection)
- **Размер**: 590,540 транзакций, 434 признака
- **Целевая переменная**: `isFraud` (мошенничество = 1, легитимная транзакция = 0)
- **Дисбаланс классов**: ~3.5% fraud (сильный дисбаланс)
- **Временной характер**: данные отсортированы по `TransactionDT`

## Задача
Бинарная классификация с **временной структурой** → используем TimeSeriesSplit для валидации.

## Подход
Сравнение 4 алгоритмов:
1. **Logistic Regression** — простая линейная модель
2. **Random Forest** — классический ансамбль деревьев
3. **LightGBM** — быстрый gradient boosting
4. **CatBoost** — автоматическая обработка категорий

## Метрика
**ROC-AUC** — основная метрика соревнования (устойчива к дисбалансу классов)



🎯 BASELINE = модель с параметрами ПО УМОЛЧАНИЮ

Зачем нужен baseline:
1. Быстрая оценка качества данных (можно ли вообще предсказывать?)
2. Точка отсчета для сравнения улучшений
3. Проверка, что предобработка работает корректно
4. Понимание, какая модель лучше подходит для задачи

Что мы НЕ делаем в baseline:
- ❌ Не настраиваем гиперпараметры (C, max_depth, learning_rate...)
- ❌ Не используем feature engineering (новые признаки)
- ❌ Не делаем feature selection (отбор признаков)
- ❌ Не используем ансамбли
- ❌ НЕ используем early stopping (это тоже оптимизация!)

Что мы делаем в baseline:
- ✅ Минимальная предобработка (заполнение пропусков, encoding)
- ✅ Параметры по умолчанию
- ✅ Честная валидация (TimeSeriesSplit)
- ✅ Сравнение разных типов моделей





In [2]:
# ============================================================================
# ИМПОРТЫ
# ============================================================================

# Системные библиотеки
import os
import gc
import warnings
from typing import List, Dict, Tuple

# Основные библиотеки для работы с данными
import numpy as np
import pandas as pd

# Визуализация
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-learn: preprocessing & imputation
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer

# Scikit-learn: models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Scikit-learn: pipeline & transformers
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Scikit-learn: validation & metrics
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import (
    roc_auc_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    confusion_matrix
)

# Gradient boosting libraries
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

# Локальные модули
from helper import *
from plots import *

# ============================================================================
# НАСТРОЙКИ
# ============================================================================

# Отключение предупреждений
warnings.filterwarnings('ignore')

# Настройки визуализации
sns.set_style("darkgrid")
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Настройки pandas
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 120)
pd.set_option('display.max_colwidth', 50)

print('✓ Окружение готово!')

✓ Окружение готово!


In [7]:
# ============================================================================
# 1. ЗАГРУЗКА И ПОДГОТОВКА
# ============================================================================

print("📂 Загрузка данных...")
train = load_data('../data/train_transaction.csv', '../data/train_identity.csv')
train = train.sort_values('TransactionDT').reset_index(drop=True)

print(f"Train shape: {train.shape}")
print(f"Fraud rate: {train['isFraud'].mean():.2%}")

X = train.drop(['isFraud', 'TransactionID'], axis=1)
y = train['isFraud']

numeric_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object']).columns.tolist()

print(f"\n📊 Числовые: {len(numeric_features)}, Категориальные: {len(categorical_features)}")

# ============================================================================
# 2. ВРЕМЕННОЕ РАЗБИЕНИЕ (БЕЗ КРОСС-ВАЛИДАЦИИ ДЛЯ BASELINE)
# ============================================================================

print("\n" + "="*100)
print("✂️ РАЗБИЕНИЕ ДАННЫХ (временное 80/20)")
print("="*100)

# Последние 20% - валидация
split_point = int(len(X) * 0.8)

X_train = X.iloc[:split_point]
X_val = X.iloc[split_point:]
y_train = y.iloc[:split_point]
y_val = y.iloc[split_point:]

print(f"Train size: {len(X_train):,} ({len(X_train)/len(X):.1%})")
print(f"Val size:   {len(X_val):,} ({len(X_val)/len(X):.1%})")
print(f"Train fraud rate: {y_train.mean():.2%}")
print(f"Val fraud rate:   {y_val.mean():.2%}")

# ============================================================================
# 3. СОЗДАНИЕ PIPELINES (БЕЗ DATA LEAKAGE!)
# ============================================================================

# 1. Pipeline для Logistic Regression
numeric_transformer_logreg = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer_logreg = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

preprocessor_logreg = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_logreg, numeric_features),
        ('cat', categorical_transformer_logreg, categorical_features)
    ])

pipeline_logreg = Pipeline(steps=[
    ('preprocessor', preprocessor_logreg),
    ('classifier', LogisticRegression(max_iter=100, random_state=42, n_jobs=-1))
])

# 2. Pipeline для Random Forest
numeric_transformer_rf = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

categorical_transformer_rf = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

preprocessor_rf = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer_rf, numeric_features),
        ('cat', categorical_transformer_rf, categorical_features)
    ])

pipeline_rf = Pipeline(steps=[
    ('preprocessor', preprocessor_rf),
    ('classifier', RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        random_state=42,
        n_jobs=-1
    ))
])

# 3. Pipeline для LightGBM
categorical_transformer_lgb = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

preprocessor_lgb = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_features),
        ('cat', categorical_transformer_lgb, categorical_features)
    ])

pipeline_lgb = Pipeline(steps=[
    ('preprocessor', preprocessor_lgb),
    ('classifier', LGBMClassifier(
        n_estimators=100,
        random_state=42,
        n_jobs=-1,
        verbose=-1
    ))
])

# 4. Pipeline для CatBoost
categorical_transformer_cat = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing'))
])

preprocessor_cat = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_features),
        ('cat', categorical_transformer_cat, categorical_features)
    ])

cat_indices = list(range(len(numeric_features), len(numeric_features) + len(categorical_features)))

pipeline_cat = Pipeline(steps=[
    ('preprocessor', preprocessor_cat),
    ('classifier', CatBoostClassifier(
        iterations=100,
        cat_features=cat_indices,
        random_state=42,
        verbose=0,
        thread_count=-1
    ))
])

print("\n" + "="*100)
print("✅ BASELINE МОДЕЛИ С ПРАВИЛЬНЫМИ PIPELINES (БЕЗ DATA LEAKAGE):")
print("="*100)
print("""
1. Logistic Regression Pipeline:
   ✓ Numeric: SimpleImputer(median) → StandardScaler
   ✓ Categorical: SimpleImputer(constant) → OrdinalEncoder(handle_unknown='use_encoded_value')
   → ColumnTransformer → LogisticRegression

2. Random Forest Pipeline:
   ✓ Numeric: SimpleImputer(median)
   ✓ Categorical: SimpleImputer(constant) → OrdinalEncoder(handle_unknown='use_encoded_value')
   → ColumnTransformer → RandomForestClassifier

3. LightGBM Pipeline:
   ✓ Numeric: passthrough (LightGBM обрабатывает NaN)
   ✓ Categorical: SimpleImputer(constant) → OrdinalEncoder(handle_unknown='use_encoded_value')
   → ColumnTransformer → LGBMClassifier

4. CatBoost Pipeline:
   ✓ Numeric: passthrough
   ✓ Categorical: SimpleImputer(constant) (CatBoost обрабатывает категории напрямую)
   → ColumnTransformer → CatBoostClassifier
""")

# ============================================================================
# 4. ОБУЧЕНИЕ И ОЦЕНКА МОДЕЛЕЙ
# ============================================================================

from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    confusion_matrix
)

models = {
    '1. Logistic Regression': pipeline_logreg,
    '2. Random Forest': pipeline_rf,
    '3. LightGBM': pipeline_lgb,
    '4. CatBoost': pipeline_cat
}

print("\n" + "="*100)
print("🚀 ОБУЧЕНИЕ BASELINE МОДЕЛЕЙ")
print("="*100)

results = {}

for model_name, pipeline in models.items():
    print(f"\n{'='*100}\n🚀 {model_name}\n{'='*100}")
    
    # Обучение
    print("⏳ Обучение...")
    pipeline.fit(X_train, y_train)
    
    # Предсказания
    y_pred_proba = pipeline.predict_proba(X_val)[:,1]
    y_pred = pipeline.predict(X_val)
    
    # Метрики
    pr_auc = average_precision_score(y_val, y_pred_proba)
    roc_auc = roc_auc_score(y_val, y_pred_proba)
    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred, zero_division=0)
    recall = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    
    print(f"\n{'-'*100}\n📊 МЕТРИКИ:\n{'-'*100}")
    print(f"PR AUC       {pr_auc:.4f}  ← ОСНОВНАЯ МЕТРИКА (для дисбаланса)")
    print(f"ROC AUC      {roc_auc:.4f}  (официальная метрика Kaggle)")
    print(f"ACCURACY     {accuracy:.4f}")
    print(f"PRECISION    {precision:.4f}")
    print(f"RECALL       {recall:.4f}")
    print(f"F1           {f1:.4f}")
    
    results[model_name] = {
        'pr_auc': pr_auc,
        'roc_auc': roc_auc,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }
    
    # Classification Report
    print(f"\n{'-'*100}\n📋 CLASSIFICATION REPORT:\n{'-'*100}")
    print(classification_report(y_val, y_pred, target_names=['Legitimate', 'Fraud']))
    
    # Confusion Matrix
    cm = confusion_matrix(y_val, y_pred)
    print(f"\n{'-'*100}\n📊 CONFUSION MATRIX:\n{'-'*100}")
    print(f"                 Predicted")
    print(f"                 Legit  Fraud")
    print(f"Actual Legit    {cm[0,0]:6d} {cm[0,1]:6d}")
    print(f"       Fraud    {cm[1,0]:6d} {cm[1,1]:6d}")
    
    # Feature importance
    trained_model = pipeline.named_steps['classifier']
    
    if hasattr(trained_model, 'feature_importances_'):
        print(f"\n{'-'*100}\n🔍 ТОП-20 ПРИЗНАКОВ:\n{'-'*100}")
        
        feature_names = numeric_features + categorical_features
        
        importance = pd.DataFrame({
            'feature': feature_names,
            'importance': trained_model.feature_importances_
        }).sort_values('importance', ascending=False).head(20)
        
        for idx, row in enumerate(importance.itertuples(), 1):
            print(f"{idx:2d}. {row.feature:30s} {row.importance:.6f}")
    
    elif hasattr(trained_model, 'coef_'):
        print(f"\n{'-'*100}\n🔍 ТОП-20 ПРИЗНАКОВ (по абсолютному значению коэффициентов):\n{'-'*100}")
        
        feature_names = numeric_features + categorical_features
        coefficients = trained_model.coef_[0]
        
        importance = pd.DataFrame({
            'feature': feature_names,
            'coefficient': coefficients,
            'abs_coefficient': np.abs(coefficients)
        }).sort_values('abs_coefficient', ascending=False).head(20)
        
        for idx, row in enumerate(importance.itertuples(), 1):
            direction = "↑ Fraud" if row.coefficient > 0 else "↓ Fraud"
            print(f"{idx:2d}. {row.feature:30s} {row.coefficient:+.4f}  ({direction})")

# ============================================================================
# 5. ИТОГОВОЕ СРАВНЕНИЕ
# ============================================================================

print(f"\n{'='*100}\n🏆 ИТОГОВОЕ СРАВНЕНИЕ\n{'='*100}")

comparison = pd.DataFrame({
    'Model': list(results.keys()),
    'PR AUC': [results[m]['pr_auc'] for m in results.keys()],
    'ROC AUC': [results[m]['roc_auc'] for m in results.keys()],
    'Precision': [results[m]['precision'] for m in results.keys()],
    'Recall': [results[m]['recall'] for m in results.keys()],
    'F1': [results[m]['f1'] for m in results.keys()]
}).sort_values('PR AUC', ascending=False)

print(f"\n{comparison.to_string(index=False)}")

print(f"\n{'='*100}")
print(f"🥇 ЛУЧШАЯ МОДЕЛЬ: {comparison.iloc[0]['Model']}")
print(f"   PR AUC:  {comparison.iloc[0]['PR AUC']:.4f} ← ОСНОВНАЯ МЕТРИКА")
print(f"   ROC AUC: {comparison.iloc[0]['ROC AUC']:.4f} (Kaggle)")
print(f"{'='*100}")

comparison.to_csv('baseline_comparison.csv', index=False)
print("\n💾 Результаты сохранены: baseline_comparison.csv")
print("\n✅ ГОТОВО!")

📂 Загрузка данных...
Загрузка данных...
Датасет загружен: (590540, 434)
Train shape: (590540, 434)
Fraud rate: 3.50%

📊 Числовые: 401, Категориальные: 31

✂️ РАЗБИЕНИЕ ДАННЫХ (временное 80/20)
Train size: 472,432 (80.0%)
Val size:   118,108 (20.0%)
Train fraud rate: 3.51%
Val fraud rate:   3.44%

✅ BASELINE МОДЕЛИ С ПРАВИЛЬНЫМИ PIPELINES (БЕЗ DATA LEAKAGE):

1. Logistic Regression Pipeline:
   ✓ Numeric: SimpleImputer(median) → StandardScaler
   ✓ Categorical: SimpleImputer(constant) → OrdinalEncoder(handle_unknown='use_encoded_value')
   → ColumnTransformer → LogisticRegression

2. Random Forest Pipeline:
   ✓ Numeric: SimpleImputer(median)
   ✓ Categorical: SimpleImputer(constant) → OrdinalEncoder(handle_unknown='use_encoded_value')
   → ColumnTransformer → RandomForestClassifier

3. LightGBM Pipeline:
   ✓ Numeric: passthrough (LightGBM обрабатывает NaN)
   ✓ Categorical: SimpleImputer(constant) → OrdinalEncoder(handle_unknown='use_encoded_value')
   → ColumnTransformer → LGBMClassi


----------------------------------------------------------------------------------------------------
📊 МЕТРИКИ:
----------------------------------------------------------------------------------------------------
PR AUC       0.4785  ← ОСНОВНАЯ МЕТРИКА (для дисбаланса)
ROC AUC      0.8659  (официальная метрика Kaggle)
ACCURACY     0.9732
PRECISION    0.7481
RECALL       0.3339
F1           0.4617

----------------------------------------------------------------------------------------------------
📋 CLASSIFICATION REPORT:
----------------------------------------------------------------------------------------------------
              precision    recall  f1-score   support

  Legitimate       0.98      1.00      0.99    114044
       Fraud       0.75      0.33      0.46      4064

    accuracy                           0.97    118108
   macro avg       0.86      0.66      0.72    118108
weighted avg       0.97      0.97      0.97    118108


-------------------------------------------


 ### 🎯 КЛЮЧЕВЫЕ МОМЕНТЫ:

- 1. CatBoost — ЕДИНСТВЕННАЯ модель, которая:
   ✓ Работает с категориями как со строками (без encoding)
   ✓ Обрабатывает NaN в категориях и числах автоматически
   ✓ Использует Ordered Target Encoding (оптимально для fraud detection)

- 2. Почему CatBoost обрабатывает пропуски лучше:
   • В числовых: NaN трактуется как отдельное значение (может быть сигналом!)
   • В категориях: NaN может означать "отсутствие данных" (тоже паттерн)
   • Модель сама решает, как использовать эту информацию

- 3. Зачем CatBoostPreprocessor:
   • Только приводит категории к типу str (технический момент)
   • НЕ заполняет пропуски, НЕ кодирует категории
   • Минимальное вмешательство = максимум для CatBoost

⚡ ПРАКТИЧЕСКИЙ СОВЕТ:

Если у вас в данных:
• Много категорий (>5 признаков)
• Высокая кардинальность (email, device_id)
• Много пропусков (>10%)

→ CatBoost с cat_features даст лучший результат, чем любая другая модель!

Для IEEE-CIS это идеальный выбор: ProductCD, card*, addr*, P_emaildomain, etc.